<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Parameterized PDE / Aero-Structure</div>

The aim of this notebook is to find the way to estimate the result of a parameterized PDE problem with a group of pre-computed results, so that the computation time can be cut down. There methods tested are as bellow:
1. [Reference solution](#ref)
3. [POD](#pod)
    1. [POD-Greddy Algorithm](#pod_ga)
    2. [POD-TensorFlow Version I](#tf1)
    3. [POD-TensorFlow Version II](#tf2)
4. [Gaussian Processe](#gp)
5. [SMT KPLS](#smt)

In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import scipy as sp
from scipy import linalg
import timeit
from sklearn.externals import joblib
import shutil
from pyDOE import lhs
import tensorflow as tf
import deepxde


from gaussian_process import GaussianProcess
import study.functions_Offline as foff
from transfert.transfert import transfert_matrix
from mesh.mesh_deformation_airbus.mesh_deformation_airbus import mesh_deformation_airbus
from VLM.VLM import VLM_study
from FEM.FEM import FEM_study
import matplotlib.pyplot as plt

import ipywidgets
from IPython.display import display

/Users/ydli/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# <a id="ref"></a> 1. Reference solution


In [3]:
def aero_struct(h_skins,h_ribs,h_spars_le,h_spars_te,b,S):
    a = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/fem_mesh.msh') 
    a.new_wing_mesh(b = b, S = S)
    #VLM meshes of the parametric wing
    vlm_mesh_file =  '../mesh/param_wing/new_VLM.msh'
    vlm_mesh_file_out =  '../mesh/param_wing/new_VLM_def.msh'
    #Beam FEM mesh of the parametric wing
    fem_mesh = '../mesh/param_wing/new_fem.msh'
    ##initialization
    #VLM_study
    alpha = 2.5
    le = [400,700]
    te = [200,500]
    my_vlm = VLM_study(vlm_mesh_file,alpha,le,te,symmetry = True, v_inf = 250.72, rho = 0.3629)
    #deformed mesh file which is the same as the non deformed one at initialization
    shutil.copyfile(vlm_mesh_file,vlm_mesh_file_out)
    #FEM_study
    E = 70*1e9
    nu = 0.3
    element_property = {'tri':{'"skins"':{'h':h_skins},'"ribs"':{'h':h_ribs},
                               '"spars_le"':{'h':h_spars_le},'"spars_te"':{'h':h_spars_te}}}
    material = {'tri':{'"skins"':{'E':E,'nu':nu},'"ribs"':{'E':E,'nu':nu},
                       '"spars_le"':{'E':E,'nu':nu},'"spars_te"':{'E':E,'nu':nu}}}
    element_type = {'tri':'DKT'}
    my_fem = FEM_study(fem_mesh,element_type,element_property,material)
    #transfert
    my_vlm.read_gmsh_mesh()
    vlm_nodes_tot = my_vlm.nodes.copy()
    vlm_nodes = vlm_nodes_tot[:,1:]
    vlm_nodes_ind = vlm_nodes_tot[:,0]
    elem_dict,element_tot = my_fem.read_mesh_file()
    fem_nodes_tot = elem_dict['nodes'].copy()
    #for the transfert we only uses the nodes that belongs to the skins
    elm_skins = elem_dict['element_sets'][1]['elements']
    node_skins = np.unique(elm_skins[:,3:6])
    node_skins = np.array(node_skins,dtype = int)
    fem_nodes = fem_nodes_tot[node_skins-1,1:]
    fem_nodes_ind = node_skins
    H = transfert_matrix(fem_nodes,vlm_nodes,function_type='thin_plate')
    ###fixed point
    err = 1.0
    tol = 1e-3
    itermax = 250
    n = 0
    u_s_0 = np.ones((len(fem_nodes_tot)*6,))*1000.0
    gamma_0 = np.ones((np.sum([my_vlm.surfaces[i]['points'].shape[0]for i in range(2)]),))*1000.0
    while n<itermax and err>tol:
        #run the VLM analysis
        my_vlm = VLM_study(vlm_mesh_file_out,alpha, le, te, symmetry = True, v_inf = 250.72, rho = 0.3629)
        gamma = my_vlm.compute_circulation()
        speed,forces = my_vlm.compute_velocities_and_forces(gamma)
        my_vlm.post_processing_gamma_reel("Param_wing/g_real",gamma)
        #compute forces at nodes
        F_a = my_vlm.compute_force_at_nodes(vlm_nodes,forces)
        #transfert this forces to the structural beam model 
        F_s = np.dot(H.T,F_a)
        #rigidity matrix
        K = my_fem.assembling_K()
        #creating the rhs for the fem study
        rhs = np.zeros((len(fem_nodes_tot)*6,))
        for i in range(3):
            rhs[6*(fem_nodes_ind-1)+i] = F_s[:,i]
        my_fem.set_rhs(rhs)    
        #Boundary condition
        ind = fem_nodes[:,1]<=0.06
        nodes_sets = [fem_nodes_ind[ind]]
        dof = [[0,1,2,3,4,5]]
        my_fem.boundary_conditions(nodes_sets,dof)
        #solving
        u_s_tot = my_fem.solve()
        my_fem.post_processing_reel(u_s_tot,"../results/Param_wing/u_real")
        #Strain and Stress
        strain_tot,stress_tot = my_fem.get_strain_and_stress(u_s_tot)
        Von_Mises_Stress = my_fem.get_Von_Mises(stress_tot)
        my_fem.post_processing_Von_Mises_reel(Von_Mises_Stress,"../results/Param_wing/result_VM")
        #print("Average Von Mises Stress (orig) = "+str(np.mean(Von_Mises_Stress))+" Pa")
        #interpolation of the displacement at skin nodes
        u_a = np.zeros((len(vlm_nodes)*6,))
        for i in range(6):
            u_a[0+i::6] = np.dot(H,u_s_tot[6*(fem_nodes_ind-1)+i])
        #vlm mesh deformation
        new_nodes = vlm_nodes_tot.copy()
        new_nodes[:,1] = new_nodes[:,1] + u_a[0::6]
        new_nodes[:,2] = new_nodes[:,2] + u_a[1::6]
        new_nodes[:,3] = new_nodes[:,3] + u_a[2::6]
        my_vlm.deformed_mesh_file(new_nodes,vlm_mesh_file_out)
        #compute relative error
        err = np.max([np.linalg.norm(u_s_0-u_s_tot)/np.linalg.norm(u_s_tot),np.linalg.norm(gamma_0-gamma)/np.linalg.norm(gamma)])
        u_s_0 = u_s_tot.copy()
        gamma_0 = gamma.copy()
        n = n+1
        #print('--------------------'+str(n)+'---------------------')
    return u_s_0, gamma_0

In [52]:
h_skins = ipywidgets.FloatSlider(value=20e-3,min=1.45e-3,max=30e-3,step=0.001,description='h_skins:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_ribs = ipywidgets.FloatSlider(value=10e-3,min=1.45e-3,max=15e-3,step=0.001,description='h_ribs:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_le = ipywidgets.FloatSlider(value=50e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_le:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_te = ipywidgets.FloatSlider(value=40e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_te:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
b = ipywidgets.FloatSlider(value=55,min=34,max=80,step=0.1,description='b:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.1f',)
s = ipywidgets.FloatSlider(value=480,min=122,max=845,step=1,description='s:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,)
ui = ipywidgets.HBox([h_skins,h_ribs,h_spars_le,h_spars_te,b,s])

def f1(h_skins,h_ribs,h_spars_le,h_spars_te,b,s):
    tic = timeit.default_timer()
    u,g=aero_struct(h_skins,h_ribs,h_spars_le,h_spars_te,b,s)
    toc = timeit.default_timer()
    time=toc-tic
    print("COMPUTATION TIME: "+str(time)+" s")
#     fig = plt.figure(figsize=(15, 10))
#     ax = fig.add_subplot(111)
#     ax.set_yticklabels(np.linspace(0,1,0.1))
#     ax.set_xticklabels(np.linspace(0,1,0.1))
#     im = ax.imshow(u, cmap=plt.cm.hot_r)
#     plt.colorbar(im)
#     plt.legend()
#     #plt.title('DeepXDE (u='+str(u)+', k='+str(k)+', ybar='+str(ybar)+')')
#     plt.show()
    #print((u, k, ybar))

out = ipywidgets.interactive_output(f1, {'h_skins':h_skins,'h_ribs':h_ribs,'h_spars_le':h_spars_le,'h_spars_te':h_spars_te,'b':b,'s':s})

display(ui, out)

Output()

The reference computation time is around 30 sec.

# <a id="pod"></a>2. POD

Proper Orthogonal Decomposition or called Principal Component Analysis (PCA) is a widely used ROM method. It converts a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables -called principal components- by using an orthogonal transformation. This transformation is defined such the first principal component has the largest variance (how much the data could variate), and the succeeding components must be orthogonal to their precedent while having the highest variance.

Rewriting the PDE problem in the form below:
$$A(\xi)X(x,y)+b(\xi)=0 $$

## <a id="pod_ga"></a> 2.1 POD-Greedy Algorithm

In this section the reduced base is computed by a greddy algorithm (which is realized by Oriol). A Greedy Algorithm is based on the estimated results of each candidate at the reduced base, in each iteration the worst point is chossen and its exact solution is used to recompute the base. 


Once the reduced base is computed, we can transfer the real PDE problem to a reduced order model. To resolve the parametric PDE problem, serveal candidate points are uniformly choosen from the parameters' domain to feed a estimateur with Kriging interpolation.

In [67]:
##PARAMETERS BOUNDS: h_skins, h_ribs, h_spars_le, h_spars_te, b, S
# h_* = thickness of different parts (m)
# b = wing span (m)
# S = wing surface (m^2)
pMin_i = np.array([0.00145,0.00145,0.00435,0.00435,34.0,122.0])
pMax_i = np.array([0.030,0.015,0.090,0.090,80.0,845.0])
pMin = np.zeros((6,1))
pMin[:,0]= pMin_i[:]
pMax = np.zeros((6,1))
pMax[:,0]= pMax_i[:]
np.save("../results/Offline/pMin",pMin)
np.save("../results/Offline/pMax",pMax)

##OTHERS PARAMETERS OF INTEREST
# alpha = angle of attack
# M_inf = Mach number
# E = Young modulus (Pa)
# nu = Poisson's ratio
# BF = distance of the fuselage over the wing (m)
# phi_d = sweep angle
# diedre_d = dihedral angle 
# n_ribs_1 = number of ribs (first sector)
# n_ribs_2 = number of ribs (second sector)
alpha = 2.5 
M_inf = 0.85
E = 70.*1e9
nu = 0.3
BF = 5.96
phi_d = 30.
diedre_d = 1.5
n_ribs_1 =  8
n_ribs_2 =  12

##FIXED PARAMETERS (Flight Altitude = 11000 meters)
# rho = density at 11000 meters (kg/m^3)
# Pressure = pressure at 11000 meters (Pa)
# adiabatic_i = adiabatic index of air
# a_speed = speed of sound at 11000 meters (m/s)
# phi = sweep angle (rad)
# diedre = dihedral angle (rad)
rho = 0.3629
Pressure = 22552.
adiabiatic_i = 1.4
a_speed = np.sqrt(adiabiatic_i*Pressure/rho)
v_inf = M_inf*a_speed
phi = phi_d*np.pi/180.
diedre = diedre_d*np.pi/180.

##CREATION OF MESH FILES VLM AND FEM
foff.create_mesh_files(n_ribs_1,n_ribs_2)

param_data = np.zeros(9)
param_data[0] = alpha
param_data[1] = v_inf
param_data[2] = rho
param_data[3] = E
param_data[4] = nu
np.save("../results/POD_G/param_data",param_data)

##GREEDY ALGORITHM
tic = timeit.default_timer()
# 0) Initialization
nSamples = 10
nIterations = nSamples-1
nCandidates = 50
nParam = len(pMin)
pCandidate = np.dot(pMin,np.ones((1,nCandidates))) + lhs(nParam,nCandidates).T*np.dot((pMax-pMin),np.ones((1,nCandidates)))
indexCand = np.zeros((nCandidates,), dtype=int)
for i in range(nCandidates):
    indexCand[i] = i
pCandMax = np.zeros((nParam,nSamples))
np.save("../results/POD_G/pCandidate",pCandidate)
np.save("../results/POD_G/nCandidates",nCandidates)
np.save("../results/POD_G/nSamples",nSamples)
# 1) Ramdomly select a first sample (in the middle of the domain)
pMiddle = (pMin+pMax)/2
param_data[5] = pMiddle[0,0]
param_data[6] = pMiddle[1,0]
param_data[7] = pMiddle[2,0]
param_data[8] = pMiddle[3,0]
b = pMiddle[4,0]
S = pMiddle[5,0]
pCandMax[:,0] = pMiddle[:,0]
# Calculating the number of nodes and initialization of samples matrices
n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(param_data)
uSamples = np.zeros((n_FEM_nodes*6,nSamples)) # POD displacement
gSamples = np.zeros((n_gamma_nodes,nSamples)) # POD circulation
# 2) Solving HDM-based problem
print ("Greedy Iteration num.: 0\n",)
uMiddle,gMiddle = foff.run_aerostruct(param_data,b,S,phi,diedre,BF,Mach=M_inf)
uSamples[:,0] = uMiddle
gSamples[:,0] = gMiddle
# 3) Building the ROB
uV,_,_ = linalg.svd(uSamples,full_matrices=False)
gV,_,_ = linalg.svd(gSamples,full_matrices=False)
uV_tr = np.transpose(uV)
gV_tr = np.transpose(gV)
# 4) Iteration loop

err_ok = 0
err_ko = 0
err_max = 0

for iIteration in range(nIterations):
    print ("Greedy Iteration num.: "+str(iIteration+1))
    # 5) Solve argmax(residual)
    maxError = 0.
    candidateMaxError = 0
    for iCandidate in indexCand:
        # 5) Loading candidates information
        param_data[5] = pCandidate[0,iCandidate]
        param_data[6] = pCandidate[1,iCandidate]
        param_data[7] = pCandidate[2,iCandidate]
        param_data[8] = pCandidate[3,iCandidate]
        b = pCandidate[4,iCandidate]
        S = pCandidate[5,iCandidate]
        ###Fixed point convergence
        err = 1.0
        err_vec = []
        tol = 5.0e-2
        itermax = 150
        n = 0
        q_0 = np.ones((nSamples,))*1.0
        g_0 = np.ones((nSamples,))*1.0
        tic1 = timeit.default_timer()
        while n < itermax and err > tol:
            ## Definition of the mesh...
            ### a) Applying the right values in the wing
            a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
            a_new.new_wing_mesh(b,S,phi,diedre,BF,Mach=M_inf)
            ### b) Defining the files used and the parameters
            vlm_mesh_file = '../mesh/param_wing/new_VLM.msh'
            if n == 0:
                vlm_mesh_file_out = '../mesh/param_wing/new_VLM_def.msh'
                shutil.copyfile(vlm_mesh_file,vlm_mesh_file_out)
            else:
                my_vlm = VLM_study(vlm_mesh_file_out,alpha = param_data[0],v_inf = param_data[1],rho = param_data[2])
                my_vlm.deformed_mesh_file(new_nodes,vlm_mesh_file_out)
            ### c) Performing the analysis and saving the results
            vlm_part = VLM_study(vlm_mesh_file_out,alpha = param_data[0],v_inf = param_data[1],rho = param_data[2])
            A = vlm_part.get_A()
            B = vlm_part.get_B()
            vlm_part.read_gmsh_mesh()
            vlm_nodes_tot = vlm_part.nodes.copy()
            
            Ar = np.dot(np.dot(gV_tr,A),gV)
            Br = np.dot(gV_tr,B)
            Ar_lu = linalg.lu_factor(Ar)
            gr = linalg.lu_solve(Ar_lu,-Br)
            g_exp = np.dot(gV,gr)
            F_s, fem_nodes, vlm_nodes = foff.get_Fs(g_exp,param_data)
            K, Fs, fem_nodes_ind = foff.get_FEM(param_data,F_s)
            Kr = np.dot(np.dot(uV_tr,K),uV)
            Fr = np.dot(uV_tr,Fs)
            q = linalg.solve(Kr,Fr)
            q_exp = np.dot(uV,q)
            ## Loop corroboration
            H = transfert_matrix(fem_nodes,vlm_nodes,function_type='thin_plate')
            q_a = np.zeros((len(vlm_nodes)*6,))
            for i in range(6):
                q_a[0+i::6] = np.dot(H,q_exp[6*(fem_nodes_ind-1)+i])
            ### VLM mesh deformation 
            new_nodes = vlm_nodes_tot.copy()
            new_nodes[:,1] = new_nodes[:,1] + q_a[0::6]
            new_nodes[:,2] = new_nodes[:,2] + q_a[1::6]
            new_nodes[:,3] = new_nodes[:,3] + q_a[2::6]
            ### Compute relative error
            err = np.max([np.linalg.norm(q_0-q)/np.linalg.norm(q),np.linalg.norm(g_0-gr)/np.linalg.norm(gr)])
            print('Error = '+str(err))
            err_vec = np.append(err_vec,err)
            q_0 = q.copy()
            g_0 = gr.copy()
            n += 1
        if n == itermax:
            err_ko += 1
            if err > err_max: err_max = err
        else:
            err_ok += 1
        # 5b) Compute error indicator
        toc1 = timeit.default_timer()
        print("Reduced convergence time = "+str(toc1-tic1)+"s")
        print("n = "+str(n))
#        plt.plot(np.linspace(1,n,n),err_vec)
#        plt.ylim(top=1e-1)
#        plt.ylim(bottom=1e-4)
#        plt.grid(True)
#        plt.show()
        
        Y1 = np.dot(K,np.dot(uV,q))- Fs
        errorIndicator = np.dot(np.transpose(Y1),Y1)
        if (iCandidate == 0 or maxError < errorIndicator):
            candidateMaxError = iCandidate
            maxError = errorIndicator
    # 6) Solving HDM-based problem for the candidate with the highest error indicator
    param_data[5] = pCandidate[0,candidateMaxError]
    param_data[6] = pCandidate[1,candidateMaxError]
    param_data[7] = pCandidate[2,candidateMaxError]
    param_data[8] = pCandidate[3,candidateMaxError]
    b = pCandidate[4,candidateMaxError]
    S = pCandidate[5,candidateMaxError]
    pCandMax[:,iIteration+1] = pCandidate[:,candidateMaxError]
    uIter,gIter = foff.run_aerostruct(param_data,b,S,phi,diedre,BF,Mach=M_inf)
    uSamples[:,iIteration+1] = uIter
    gSamples[:,iIteration+1] = gIter
    # 7) Building the ROB
    uV,_,_ = linalg.svd(uSamples,full_matrices=False)
    gV,_,_ = linalg.svd(gSamples,full_matrices=False)
    uV_tr = np.transpose(uV)
    gV_tr = np.transpose(gV)
    ##UPDATING indexCand: The current index must be removed!
    xC = 0
    while True:
        if indexCand[xC] == candidateMaxError:
            break
        xC += 1
    indexCand = np.delete(indexCand,xC)

##SAVE OFFLINE DATA
u_name_V = "../POD_G/Offline/uV"
g_name_V = "../POD_G/Offline/gV"
np.save(u_name_V,uV)
np.save(g_name_V,gV)
toc = timeit.default_timer()
print("TOTAL COMPUTATION TIME: "+str((toc-tic)/60.)+" min")
print("total ok = "+str(err_ok))
print("total ko = "+str(err_ko))

##KRIGING
# Initialization
tic = timeit.default_timer()
U_comp,G_comp,pCandMax = foff.kriging_extended_B(uSamples,gSamples,pCandMax,pMin,pMax, nSamples, param_data, uV, gV, phi, diedre, BF, M_inf)
np.save("../results/POD_G/U_comp",U_comp)
np.save("../results/POD_G/G_comp",G_comp)
# Weight calculations
nLHS = len(U_comp[0,:])
a_kri = np.zeros((nLHS,nSamples))
b_kri = np.zeros((nLHS,nSamples))
for i in range(nSamples):
    for j in range(nLHS):
        a_kri[j,i] = np.dot(U_comp[:,j].T,uV[:,i])
        b_kri[j,i] = np.dot(G_comp[:,j].T,gV[:,i])
np.save("../results/POD_G/a_kri",a_kri)
np.save("../results/POD_G/b_kri",b_kri)
# Trained solution
mean = "constant"
covariance = "squared_exponential"
theta_U = np.array([100000.0]*6)
theta_L = np.array([0.001]*6)
theta_0 = np.array([1.0]*6)
for i in range(nSamples):
    GP_u = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_u.fit(pCandMax.T, a_kri[:,i])
    GP_g = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_g.fit(pCandMax.T, b_kri[:,i])
    joblib.dump(GP_u, "../results/POD_G/GP_alpha_"+str(i)+".pkl")
    joblib.dump(GP_g, "../results/POD_G/GP_beta_"+str(i)+".pkl")
    
toc = timeit.default_timer()
print("KRIGING COMPUTATION TIME: "+str((toc-tic)/60.)+" min")

Greedy Iteration num.: 0



/Users/ydli/Desktop/SFE/Codes/aero_comp/Code/transfert/rbf_poly.py:88: RuntimeWarning: divide by zero encountered in log
  result = r**2 * log(r)
/Users/ydli/Desktop/SFE/Codes/aero_comp/Code/transfert/rbf_poly.py:88: RuntimeWarning: invalid value encountered in multiply
  result = r**2 * log(r)


Greedy Iteration num.: 1
Error = 4.060834285486118
Error = 0.010212318493480503
Reduced convergence time = 11.686410401001922s
n = 2
Error = 20.620449205019316
Error = 0.0016181945005381028
Reduced convergence time = 11.894107059008093s
n = 2
Error = 7.045693624911206
Error = 0.009256891033925447
Reduced convergence time = 11.773356420002528s
n = 2
Error = 4.800469903429493
Error = 0.010329284810550614
Reduced convergence time = 11.836155204990064s
n = 2
Error = 38.812466725801436
Error = 0.005420340140532125
Reduced convergence time = 11.81630478000443s
n = 2
Error = 18.414680307823616
Error = 0.004471178760783161
Reduced convergence time = 11.905239629006246s
n = 2
Error = 52.81649525861641
Error = 0.0009991945643311954
Reduced convergence time = 11.797397895992617s
n = 2
Error = 6.323517825063549
Error = 0.012076585648541645
Reduced convergence time = 11.774357989008422s
n = 2
Error = 5.982117839317013
Error = 0.011748056067597289
Reduced convergence time = 11.874443680993863s
n = 2

Error = 0.05431383231811403
Error = 0.05431167267601649
Error = 0.05430962336973048
Error = 0.05430767880401382
Error = 0.05430583366278623
Error = 0.05430408289608998
Error = 0.0543024217072665
Error = 0.054300845539736735
Error = 0.05429935006713368
Error = 0.054297931179376105
Error = 0.05429658497447925
Error = 0.05429530774673627
Error = 0.05429409597811962
Error = 0.054292946327809705
Error = 0.05429185562448302
Error = 0.05429082085713482
Error = 0.05428983916758714
Error = 0.05428890784260881
Error = 0.05428802430665861
Error = 0.054287186115155296
Error = 0.05428639094768761
Error = 0.05428563660192675
Error = 0.05428492098786095
Error = 0.05428424212141878
Error = 0.05428359812041268
Error = 0.05428298719792151
Error = 0.054282407658583616
Error = 0.0542818578937944
Error = 0.054281336375990814
Error = 0.054280841657116906
Error = 0.054280372362124504
Error = 0.054279927186715766
Error = 0.054279504893350934
Error = 0.05427910430695284
Error = 0.05427872431399076
Error = 0.05

Error = 0.09365469949592557
Error = 0.09365424647990903
Error = 0.09365383213863256
Error = 0.0936534531794748
Error = 0.09365310658916946
Error = 0.0936527896086994
Error = 0.09365249971507983
Error = 0.0936522345966129
Error = 0.09365199214044917
Error = 0.09365177041281567
Error = 0.09365156764407931
Error = 0.09365138221430629
Error = 0.09365121264430985
Error = 0.09365105757705079
Error = 0.09365091577528567
Error = 0.09365078610574783
Error = 0.09365066752949028
Error = 0.09365055909722489
Error = 0.0936504599467717
Error = 0.09365036927705558
Error = 0.09365028637064803
Error = 0.09365021055607352
Error = 0.09365014123287073
Error = 0.0936500778435401
Error = 0.09365001987651718
Error = 0.09364996687439452
Error = 0.09364991840912144
Error = 0.0936498740895091
Error = 0.09364983356864855
Error = 0.09364979651349388
Error = 0.09364976262984341
Error = 0.09364973164881447
Error = 0.09364970332246537
Error = 0.09364967741751133
Error = 0.09364965373218669
Error = 0.0936496320751853

...

Error = 0.09042028466993501
Error = 0.09043735248837101
Error = 0.09044918312399519
Error = 0.09045723925226876
Error = 0.09046252408917554
Error = 0.090465632086708
Error = 0.09046678643227839
Error = 0.09046586453859122
Error = 0.09046241247292988
Error = 0.09045564904382015
Error = 0.09044446004210424
Error = 0.09042738293667654
Error = 0.09040258215884295
Error = 0.09036781498310006
Error = 0.09032038794153514
Error = 0.09025710371489874
Error = 0.09017419854804697
Error = 0.09006727048362152
Error = 0.08993119914041262
Error = 0.08976005844465697
Error = 0.08954702472530042
Error = 0.08928428399448839
Error = 0.08896294412926536
Error = 0.08857296012443527
Error = 0.08810308360254071
Error = 0.08754085129428317
Error = 0.08687263098508072
Error = 0.08608374699800493
Error = 0.08515870987426649
Error = 0.0840815753058244
Error = 0.08283645408632402
Error = 0.0814081861271223
Error = 0.07978317596452807
Error = 0.07795036407934747
Error = 0.07590227888195912
Error = 0.07363608227691

Error = 0.14874419825224522
Error = 0.14605228732018882
Error = 0.14238877690207039
Error = 0.13759744167644913
Error = 0.1315723400467987
Error = 0.12429350404121652
Error = 0.11585419535524694
Error = 0.10646649967629991
Error = 0.09643810375143308
Error = 0.08612531741943438
Error = 0.0758781327552458
Error = 0.06599496841584408
Error = 0.05669754250601744
Error = 0.048126327450779396
Reduced convergence time = 216.44663997698808s
n = 36
Error = 1.5782252501268275
Error = 0.11967965712183572
Error = 0.11964173224289151
Error = 0.11953287879911363
Error = 0.11936726574770044
Error = 0.1191550524523233
Error = 0.11890312227013561
Error = 0.11861567457213328
Error = 0.11829469045736474
Error = 0.11794028972695836
Error = 0.11755099630833649
Error = 0.11712392772497761
Error = 0.11665492222762656
Error = 0.11613861534469286
Error = 0.11556847613768402
Error = 0.11493681252346308
Error = 0.11423475475399855
Error = 0.1134522266094338
Error = 0.1125779151199238
Error = 0.11159925175100356

Error = 0.09266227133624842
Error = 0.09381283178380304
Error = 0.09506858930643365
Error = 0.09644165978066654
Error = 0.09794588905467559
Error = 0.09959724303925505
Error = 0.10141428739938657
Error = 0.10341878490068472
Error = 0.10563644795506925
Error = 0.10809789716291907
Error = 0.11083989526445977
Error = 0.11390695225235026
Error = 0.11735343480529904
Error = 0.1212463664179778
Error = 0.12566917993660706
Error = 0.1307267891279745
Error = 0.136552486195256
Error = 0.14331734176640384
Error = 0.1512429328390769
Error = 0.16061816262609926
Error = 0.17182005143618456
Error = 0.185334805134774
Error = 0.20176370172829827
Error = 0.2217627509418854
Error = 0.24576591992186014
Error = 0.27311224183717825
Error = 0.29995435359777517
Error = 0.3165318531732858
Error = 0.3100125397903434
Error = 0.27817288111820115
Error = 0.23399220988057465
Error = 0.19092928666969683
Error = 0.15460973985557463
Error = 0.1255126044375934
Error = 0.10243882906204861
Error = 0.08401750609182443
Err

Error = 0.06883299736027615
Error = 0.0674067393198856
Error = 0.06590414089441098
Error = 0.06432579424882587
Error = 0.0626731522503744
Error = 0.06094859919750319
Error = 0.05915550324238954
Error = 0.05729824531934393
Error = 0.05538221997331348
Error = 0.05341380451762215
Error = 0.05140029445130582
Error = 0.049349804951536215
Reduced convergence time = 299.0054186100024s
n = 49
Error = 3.507476461286999
Error = 0.0059982683169629665
Reduced convergence time = 11.96884765897994s
n = 2
Error = 3.819842035922536
Error = 0.02974221864895714
Reduced convergence time = 12.072352835006313s
n = 2


FileNotFoundError: [Errno 2] No such file or directory: '../POD_G/Offline/uV.npy'

In [77]:
np.save("../results/POD_G/uSamples",uSamples)
np.save("../results/POD_G/gSamples",gSamples)

The offline computation takes around 7h30.

The online computation takes less than 1 sec. And it cut down around 95% of the calculation time comparing with the reference result.

In [78]:
h_skins = ipywidgets.FloatSlider(value=20e-3,min=1.45e-3,max=30e-3,step=0.001,description='h_skins:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_ribs = ipywidgets.FloatSlider(value=10e-3,min=1.45e-3,max=15e-3,step=0.001,description='h_ribs:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_le = ipywidgets.FloatSlider(value=50e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_le:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_te = ipywidgets.FloatSlider(value=40e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_te:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
b = ipywidgets.FloatSlider(value=55,min=34,max=80,step=0.1,description='b:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.1f',)
S = ipywidgets.FloatSlider(value=480,min=122,max=845,step=1,description='S:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,)
ui = ipywidgets.HBox([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])

def f3(h_skins,h_ribs,h_spars_le,h_spars_te,b,S):
    #tic = timeit.default_timer()
    nSamples = 10
    param_data = np.load("../results/POD_G/param_data.npy")
    uV = np.load("../results/POD_G/uV.npy")
    gV = np.load("../results/POD_G/gV.npy")

#     h_skins = 0.02
#     h_ribs = 0.01
#     h_spars_le = 0.05
#     h_spars_te = 0.04
#     b = 55.
#     S = 480.

    x_conf = np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])
    pC = np.zeros((6))
    pC[0] = param_data[5] = h_skins
    pC[1] = param_data[6] = h_ribs
    pC[2] = param_data[7] = h_spars_le
    pC[3] = param_data[8] = h_spars_te
    pC[4] = b
    pC[5] = S
    # 1) Kriging prediction of the interested point
    tic = timeit.default_timer()
    u_mp = np.zeros((1,nSamples))
    u_vp = np.zeros((1,nSamples))
    gamma_mp = np.zeros((1,nSamples))
    gamma_vp = np.zeros((1,nSamples))
    for i in range(nSamples):
        # *Loading Kriging data
        name_a = joblib.load("../results/POD_G/GP_alpha_"+str(i)+".pkl")
        name_b = joblib.load("../results/POD_G/GP_beta_"+str(i)+".pkl")
        # **Prediction
        u_mp[0,i], u_vp[0,i] = name_a.predict(x_conf.reshape((1,6)), eval_MSE=True)
        gamma_mp[0,i], gamma_vp[0,i] = name_b.predict(x_conf.reshape((1,6)), eval_MSE=True)
    u_mean = 0
    u_var2 = 0
    gamma_mean = 0
    gamma_var2 = 0
    for i in range(nSamples):
        u_mean += np.dot(u_mp[0,i],uV[:,i])
        u_var2 += np.dot(u_vp[0,i]**2,uV[:,i]**2)
        gamma_mean += np.dot(gamma_mp[0,i],gV[:,i])
        gamma_var2 += np.dot(gamma_vp[0,i]**2,gV[:,i]**2)
    # 2) Calculation of u_est and g_est
    u_est = u_mean
    u_est1 = u_mean+3.0*np.sqrt(u_var2)
    u_est2 = u_mean-3.0*np.sqrt(u_var2)
    g_est = gamma_mean
    g_est1 = gamma_mean+3.0*np.sqrt(gamma_var2)
    g_est2 = gamma_mean-3.0*np.sqrt(gamma_var2)
    toc = timeit.default_timer()
    print("ONLINE COMPUTATION TIME: "+str(toc-tic)+" s")
    # 3) Publication of the results: u_est, g_est
    a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
    a_new.new_wing_mesh(b,S)
    vlm_mesh = '../mesh/param_wing/new_VLM.msh'
    fem_mesh = '../mesh/param_wing/new_FEM.msh'
    n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
    # 3a) Computation of strains and stress
    element_property,material,element_type = foff.create_dict(param_data)
    my_fem = FEM_study(fem_mesh,element_type,element_property,material)
    my_fem.read_mesh_file()
    strain_dict, stress_dict = my_fem.get_strain_and_stress(u_est)
    # 3b) u_est
    my_fem.post_processing(u_est,"../results/Param_wing/u_est_tf1")
    my_fem.post_processing_var1(u_est1,"../results/Param_wing/u_est1_tf1")
    my_fem.post_processing_var2(u_est2,"../results/Param_wing/u_est2_tf1")
    ## 3c) Von Mises stress
    Von_Mises_Stress_r = my_fem.get_Von_Mises(stress_dict)
    my_fem.post_processing_Von_Mises(Von_Mises_Stress_r,'../results/param_wing/result_VM_iter_tf1')
    print("Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress_r))+" Pa")
    # 3d) g_est
    my_vlm = VLM_study(vlm_mesh)
    my_vlm.post_processing_gamma('Param_wing/g_est_tf1',g_est)
    my_vlm.post_processing_gamma_var1('Param_wing/g_est1_tf1',g_est1)
    my_vlm.post_processing_gamma_var2('Param_wing/g_est2_tf1',g_est2)
    
    # Reference
    tic = timeit.default_timer()
    u,g=aero_struct(h_skins,h_ribs,h_spars_le,h_spars_te,b,S)
    toc = timeit.default_timer()
    print("REFERENCE COMPUTATION TIME: "+str(toc-tic)+" s")
#     a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
#     a_new.new_wing_mesh(b,S)
#     vlm_mesh = '../mesh/param_wing/new_VLM.msh'
#     fem_mesh = '../mesh/param_wing/new_FEM.msh'
#     n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
#     # 3a) Computation of strains and stress
#     element_property,material,element_type = foff.create_dict(param_data)
#     my_fem = FEM_study(fem_mesh,element_type,element_property,material)
#     my_fem.read_mesh_file()
    strain_dict, stress_dict = my_fem.get_strain_and_stress(u)
    ## 3c) Von Mises stress
    Von_Mises_Stress = my_fem.get_Von_Mises(stress_dict)
    #my_fem.post_processing_Von_Mises(Von_Mises_Stress,'../results/param_wing/result_VM_iter')
    print("Reference Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress))+" Pa")
    
    Target=u_est
    res=u
    z1 = np.polyfit(Target, res, 1)
    f1 = np.poly1d(z1)
    plt.scatter(Target,res,color='r',label='Data')
    plt.plot(np.unique(Target),f1(np.unique(Target)),label='F')
    plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
    plt.title('POD U')
    plt.xlabel('Target')
    plt.ylabel('Output~='+str(z1[0])+'*Target+'+str(z1[1]))
    plt.legend()
    plt.show()
    
    Target=g_est
    res=g
    z2 = np.polyfit(Target, res, 1)
    f2 = np.poly1d(z2)
    plt.scatter(Target,res,color='r',label='Data')
    plt.plot(np.unique(Target),f2(np.unique(Target)),label='F')
    plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
    plt.title('POD G')
    plt.xlabel('Target')
    plt.ylabel('Output~='+str(z2[0])+'*Target+'+str(z2[1]))
    plt.legend()
    plt.show()
    
    rmse_u=np.sqrt(((u_est - u) ** 2).mean())
    print("RMSE U: %f" %rmse_u)
    rmse_g=np.sqrt(((g_est - g) ** 2).mean())
    print("RMSE G: %f" %rmse_g)
    #rmse_s=np.sqrt(((Von_Mises_Stress_r - Von_Mises_Stress) ** 2).mean())
    #print("RMSE Von Mises Stress: %f" %rmse_s)
    #error=(np.mean(Von_Mises_Stress_r)-np.mean(Von_Mises_Stress_r))
    error= np.linalg.norm(Von_Mises_Stress_r-Von_Mises_Stress)/np.linalg.norm(Von_Mises_Stress)
    print("Von Mises Stress Error: %f" %error)

out = ipywidgets.interactive_output(f3, {'h_skins':h_skins,'h_ribs':h_ribs,'h_spars_le':h_spars_le,'h_spars_te':h_spars_te,'b':b,'S':S})

display(ui, out)

Output()

## <a id="tf1"></a> 2.2 POD with TensorFlow - Gaussian Process



In [32]:
# Pre-computation of candidate points
tic = timeit.default_timer()
alpha = 2.5 
M_inf = 0.85
E = 70.*1e9
nu = 0.3
BF = 5.96
phi_d = 30.
diedre_d = 1.5
n_ribs_1 =  8
n_ribs_2 =  12

rho = 0.3629
Pressure = 22552.
adiabiatic_i = 1.4
a_speed = np.sqrt(adiabiatic_i*Pressure/rho)
v_inf = M_inf*a_speed
phi = phi_d*np.pi/180.
diedre = diedre_d*np.pi/180.

foff.create_mesh_files(n_ribs_1,n_ribs_2)

param_data = np.zeros(9)
param_data[0] = alpha
param_data[1] = v_inf
param_data[2] = rho
param_data[3] = E
param_data[4] = nu

pComp = np.load("../results/Offline/pCandidate.npy")
nComp=pComp.shape[1]
uComp = np.zeros((2988,nComp))
gComp = np.zeros((627,nComp))
count = 0

for i in range(nComp):
    print(pComp[:,count])
    param_data[5] = pComp[0,count]
    param_data[6] = pComp[1,count]
    param_data[7] = pComp[2,count]
    param_data[8] = pComp[3,count]
    b = pComp[4,count]
    S = pComp[5,count]
    u,g = foff.run_aerostruct(param_data,b,S,phi,diedre,BF,Mach=M_inf)
    #print(sp.linalg.solve(A,b).shape)
    #print(wComp[:,count])
    uComp[:,count] = u
    gComp[:,count] = g
    count += 1  
np.save("../results/Offline/uComp",uComp)
np.save("../results/Offline/gComp",gComp)
toc = timeit.default_timer()
print("OFFLINE COMPUTATION TIME: "+str((toc-tic)/60.)+" min")

[7.27395915e-03 2.08584933e-03 5.98703543e-02 1.68440274e-02
 6.11015410e+01 2.64678973e+02]
[4.74096357e-03 4.43680707e-03 1.66396430e-02 1.03249642e-02
 4.46319101e+01 1.62360853e+02]
[8.97661742e-03 9.72282324e-03 4.07844233e-02 2.83927514e-02
 6.13195988e+01 2.96227326e+02]
[1.03628902e-02 9.47026217e-03 1.19240383e-02 4.86629803e-02
 7.47338435e+01 2.67160251e+02]
[2.92070304e-02 7.74897450e-03 5.23290805e-02 7.27709995e-02
 6.77826097e+01 1.48236005e+02]
[4.24804914e-03 9.55422712e-03 6.59844053e-02 7.73913929e-02
 5.24478404e+01 6.54927953e+02]
[1.79231181e-02 1.80811429e-03 8.56220499e-02 6.82744666e-02
 5.28057879e+01 2.77584217e+02]
[2.22204255e-02 9.87957930e-03 5.21848835e-02 7.10860329e-02
 4.89231893e+01 3.11234295e+02]
[2.17034545e-03 8.48939350e-03 8.90184859e-03 8.77811553e-02
 5.50914589e+01 5.72475634e+02]
[1.14809571e-02 5.68869541e-03 7.37883991e-02 5.59216587e-02
 6.48000019e+01 4.49435901e+02]
[2.79231503e-03 3.02484838e-03 5.68478444e-03 3.12328536e-02
 6.246973

[8.40904181e-03 1.33260408e-02 8.87121691e-02 5.79847889e-02
 4.68121569e+01 3.84915353e+02]
[2.66463840e-02 6.28090345e-03 2.57038748e-02 8.92671495e-02
 6.57405909e+01 5.57355586e+02]
[2.39526561e-02 2.70287991e-03 7.83064881e-03 3.85113534e-02
 5.82120933e+01 3.33583900e+02]
[5.73315059e-03 7.59870660e-03 6.71546743e-02 5.52459339e-03
 4.77005150e+01 4.23575296e+02]
[5.22798157e-03 6.78670875e-03 5.53526615e-02 6.17428732e-02
 4.65431772e+01 3.09063340e+02]
[1.31303927e-02 1.15936144e-02 4.27166255e-02 1.47464101e-02
 5.48501093e+01 4.80863952e+02]
[2.94919023e-02 1.21229531e-02 4.78661827e-02 6.54918197e-02
 5.42214512e+01 4.78255613e+02]
[8.59258301e-03 9.15874589e-03 8.46998602e-02 3.76443877e-02
 5.95870067e+01 7.87303486e+02]
[6.09785320e-03 1.08525050e-02 8.03723727e-02 8.40770184e-02
 4.78953558e+01 3.91833347e+02]
[1.10456624e-02 1.22939579e-02 4.44568880e-02 4.51843418e-03
 7.18040457e+01 2.47848402e+02]
[2.21554248e-02 6.06530024e-03 8.62128637e-02 8.63664825e-02
 4.283526

In [52]:
tic = timeit.default_timer()
uComp=np.load("../results/Offline/uComp.npy")
gComp=np.load("../results/Offline/gComp.npy")
s_u,u_u,v_u=tf.svd(uComp)
sigma_u=tf.diag(s_u)
s_g,u_g,v_g=tf.svd(gComp)
sigma_g=tf.diag(s_g)
with tf.Session() as sess:
    s_u,u_u,v_u=sess.run([s_u,u_u,v_u])
    s_g,u_g,v_g=sess.run([s_g,u_g,v_g])

V_u=u_u[:,:50]
V_g=u_g[:,:50]

pCand=np.load("../results/Offline/pCandidate.npy")
nSamples=50

#U_comp,G_comp,pCand = foff.kriging_extended_B(uSamples,gSamples,pCand,pMin,pMax, nSamples, param_data, V_u, V_g, phi, diedre, BF, M_inf)
#np.save("../results/Offline/U_comp",U_comp)
#np.save("../results/Offline/G_comp",G_comp)
# Weight calculations
nLHS = len(uComp[0,:])
a_kri = np.zeros((nLHS,nSamples))
b_kri = np.zeros((nLHS,nSamples))
for i in range(nSamples):
    for j in range(nLHS):
        a_kri[j,i] = np.dot(uComp[:,j].T,V_u[:,i])
        b_kri[j,i] = np.dot(gComp[:,j].T,V_g[:,i])
np.save("../results/Offline/a_kri_tf1",a_kri)
np.save("../results/Offline/b_kri_tf1",b_kri)
# Trained solution
mean = "constant"
covariance = "squared_exponential"
theta_U = np.array([100000.0]*6)
theta_L = np.array([0.001]*6)
theta_0 = np.array([1.0]*6)
for i in range(nSamples):
    GP_u = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_u.fit(pCand.T, a_kri[:,i])
    GP_g = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_g.fit(pCand.T, b_kri[:,i])
    joblib.dump(GP_u, "../results/Offline/GP_alpha_tf1_"+str(i)+".pkl")
    joblib.dump(GP_g, "../results/Offline/GP_beta_tf1_"+str(i)+".pkl")

np.save("../results/Offline/V_u_tf1",V_u)
np.save("../results/Offline/V_g_tf1",V_g)
toc = timeit.default_timer()
print("TRAINING TIME: "+str((toc-tic)/60.)+" min")

TRAINING TIME: 0.8292292442332837 min


Without the iterations for greedy algorithm, the training time is mostly used to compute the reference solution, and the total computation time of the offline step is around 1h20.


The online computation takes less than 1 sec. And it cut down around 99% of the calculation time comparing with the reference result. 

Since the reduced base is chossen by limiting the information loss, the prediction results can be a little bit better than the POD with Greedy Algorithm. 

In [73]:
h_skins = ipywidgets.FloatSlider(value=20e-3,min=1.45e-3,max=30e-3,step=0.001,description='h_skins:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_ribs = ipywidgets.FloatSlider(value=10e-3,min=1.45e-3,max=15e-3,step=0.001,description='h_ribs:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_le = ipywidgets.FloatSlider(value=50e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_le:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_te = ipywidgets.FloatSlider(value=40e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_te:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
b = ipywidgets.FloatSlider(value=55,min=34,max=80,step=0.1,description='b:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.1f',)
S = ipywidgets.FloatSlider(value=480,min=122,max=845,step=1,description='S:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,)
ui = ipywidgets.HBox([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])

def f3(h_skins,h_ribs,h_spars_le,h_spars_te,b,S):
    #tic = timeit.default_timer()
    nSamples = 10
    param_data = np.load("../results/Offline/param_data.npy")
    uV = np.load("../results/Offline/V_u_tf1.npy")[:,:10]
    gV = np.load("../results/Offline/V_g_tf1.npy")[:,:10]

#     h_skins = 0.02
#     h_ribs = 0.01
#     h_spars_le = 0.05
#     h_spars_te = 0.04
#     b = 55.
#     S = 480.

    x_conf = np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])
    pC = np.zeros((6))
    pC[0] = param_data[5] = h_skins
    pC[1] = param_data[6] = h_ribs
    pC[2] = param_data[7] = h_spars_le
    pC[3] = param_data[8] = h_spars_te
    pC[4] = b
    pC[5] = S
    # 1) Kriging prediction of the interested point
    tic = timeit.default_timer()
    u_mp = np.zeros((1,nSamples))
    u_vp = np.zeros((1,nSamples))
    gamma_mp = np.zeros((1,nSamples))
    gamma_vp = np.zeros((1,nSamples))
    for i in range(nSamples):
        # *Loading Kriging data
        name_a = joblib.load("../results/Offline/GP_alpha_tf1_"+str(i)+".pkl")
        name_b = joblib.load("../results/Offline/GP_beta_tf1_"+str(i)+".pkl")
        # **Prediction
        u_mp[0,i], u_vp[0,i] = name_a.predict(x_conf.reshape((1,6)), eval_MSE=True)
        gamma_mp[0,i], gamma_vp[0,i] = name_b.predict(x_conf.reshape((1,6)), eval_MSE=True)
    u_mean = 0
    u_var2 = 0
    gamma_mean = 0
    gamma_var2 = 0
    for i in range(nSamples):
        u_mean += np.dot(u_mp[0,i],uV[:,i])
        u_var2 += np.dot(u_vp[0,i]**2,uV[:,i]**2)
        gamma_mean += np.dot(gamma_mp[0,i],gV[:,i])
        gamma_var2 += np.dot(gamma_vp[0,i]**2,gV[:,i]**2)
    # 2) Calculation of u_est and g_est
    u_est = u_mean
    u_est1 = u_mean+3.0*np.sqrt(u_var2)
    u_est2 = u_mean-3.0*np.sqrt(u_var2)
    g_est = gamma_mean
    g_est1 = gamma_mean+3.0*np.sqrt(gamma_var2)
    g_est2 = gamma_mean-3.0*np.sqrt(gamma_var2)
    toc = timeit.default_timer()
    print("ONLINE COMPUTATION TIME: "+str(toc-tic)+" s")
    # 3) Publication of the results: u_est, g_est
    a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
    a_new.new_wing_mesh(b,S)
    vlm_mesh = '../mesh/param_wing/new_VLM.msh'
    fem_mesh = '../mesh/param_wing/new_FEM.msh'
    n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
    # 3a) Computation of strains and stress
    element_property,material,element_type = foff.create_dict(param_data)
    my_fem = FEM_study(fem_mesh,element_type,element_property,material)
    my_fem.read_mesh_file()
    strain_dict, stress_dict = my_fem.get_strain_and_stress(u_est)
    # 3b) u_est
    my_fem.post_processing(u_est,"../results/Param_wing/u_est_tf1")
    my_fem.post_processing_var1(u_est1,"../results/Param_wing/u_est1_tf1")
    my_fem.post_processing_var2(u_est2,"../results/Param_wing/u_est2_tf1")
    ## 3c) Von Mises stress
    Von_Mises_Stress_r = my_fem.get_Von_Mises(stress_dict)
    my_fem.post_processing_Von_Mises(Von_Mises_Stress_r,'../results/param_wing/result_VM_iter_tf1')
    print("Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress_r))+" Pa")
    # 3d) g_est
    my_vlm = VLM_study(vlm_mesh)
    my_vlm.post_processing_gamma('Param_wing/g_est_tf1',g_est)
    my_vlm.post_processing_gamma_var1('Param_wing/g_est1_tf1',g_est1)
    my_vlm.post_processing_gamma_var2('Param_wing/g_est2_tf1',g_est2)
    
    # Reference
    tic = timeit.default_timer()
    u,g=aero_struct(h_skins,h_ribs,h_spars_le,h_spars_te,b,S)
    toc = timeit.default_timer()
    print("REFERENCE COMPUTATION TIME: "+str(toc-tic)+" s")
#     a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
#     a_new.new_wing_mesh(b,S)
#     vlm_mesh = '../mesh/param_wing/new_VLM.msh'
#     fem_mesh = '../mesh/param_wing/new_FEM.msh'
#     n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
#     # 3a) Computation of strains and stress
#     element_property,material,element_type = foff.create_dict(param_data)
#     my_fem = FEM_study(fem_mesh,element_type,element_property,material)
#     my_fem.read_mesh_file()
    strain_dict, stress_dict = my_fem.get_strain_and_stress(u)
    ## 3c) Von Mises stress
    Von_Mises_Stress = my_fem.get_Von_Mises(stress_dict)
    #my_fem.post_processing_Von_Mises(Von_Mises_Stress,'../results/param_wing/result_VM_iter')
    print("Reference Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress))+" Pa")
    
    Target=u_est
    res=u
    z1 = np.polyfit(Target, res, 1)
    f1 = np.poly1d(z1)
    plt.scatter(Target,res,color='r',label='Data')
    plt.plot(np.unique(Target),f1(np.unique(Target)),label='F')
    plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
    plt.title('POD_tf U')
    plt.xlabel('Target')
    plt.ylabel('Output~='+str(z1[0])+'*Target+'+str(z1[1]))
    plt.legend()
    plt.show()
    
    Target=g_est
    res=g
    z2 = np.polyfit(Target, res, 1)
    f2 = np.poly1d(z2)
    plt.scatter(Target,res,color='r',label='Data')
    plt.plot(np.unique(Target),f2(np.unique(Target)),label='F')
    plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
    plt.title('POD_tf G')
    plt.xlabel('Target')
    plt.ylabel('Output~='+str(z2[0])+'*Target+'+str(z2[1]))
    plt.legend()
    plt.show()
    
    rmse_u=np.sqrt(((u_est - u) ** 2).mean())
    print("RMSE U: %f" %rmse_u)
    rmse_g=np.sqrt(((g_est - g) ** 2).mean())
    print("RMSE G: %f" %rmse_g)
    #rmse_s=np.sqrt(((Von_Mises_Stress_r - Von_Mises_Stress) ** 2).mean())
    #print("RMSE Von Mises Stress: %f" %rmse_s)
    #error=(np.mean(Von_Mises_Stress_r)-np.mean(Von_Mises_Stress_r))
    error= np.linalg.norm(Von_Mises_Stress_r-Von_Mises_Stress)/np.linalg.norm(Von_Mises_Stress)
    print("Von Mises Stress Error: %f" %error)

out = ipywidgets.interactive_output(f3, {'h_skins':h_skins,'h_ribs':h_ribs,'h_spars_le':h_spars_le,'h_spars_te':h_spars_te,'b':b,'S':S})

display(ui, out)

Output()

## <a id="tf2"></a> 3.3 POD with TensorFlow - Similarity Criterion Estimation

In this section, the resolution of a parameterized PDE problem is completly realized by TensorFlow. First, at the training step, a reduced base is constructed based on 1000 observations (the PDE results of 1000 groups of parameters $\mathbf{\xi}$):

$$\mathbf{X}=[X_0,...,X_{999}]=U\Sigma V$$

The corresponding coefficients to transport the reduced base results to real base results are also computed:

$$\mathbf{X_r}=[X_{r0},...,X_{r999}]=U^{-1}\mathbf{X} \textrm{, where } X_i=X_{ri}U \textrm{ for } 0\leq i<1000$$

In this method, only the parameters and the computational results in the observation set are needed. And it's not necessary to know the structure of the real PDE system. It means that we can realize the estimation of a black box system just by a group of observations. 

Since the results are computed based on the existing observations, the number of observations has a great influence on the prediction precision. And the prediction works very well when the new parameters are closed to the parameters in the observation set. However, when these parameters are not in the observation set, the precision becomes worse.

For a group of parameters $\xi^*$ which is not in the observation set, the coefficient $X_r^*$ is estimated by a similarity criterion. And here the cosine similarity index is used:

$$X_r^*=X_{r\xi_k}+(X_{r\xi_{k+1}}-X_{r\xi_k})\frac{\xi^*-\xi_k}{\xi_{k+1}-\xi_k} \textrm{, where } \xi_k<\xi^*<\xi_{k+1}$$

# <a id="gp"></a> 4. Gaussian Process

In [2]:
uComp=np.load("../results/Offline/uComp.npy")
gComp=np.load("../results/Offline/gComp.npy")
pComp=np.load("../results/Offline/pComp.npy")

In [55]:
tic = timeit.default_timer()
mean = "constant"
covariance = "squared_exponential"
theta_U = np.array([100000.0]*6)
theta_L = np.array([0.001]*6)
theta_0 = np.array([1.0]*6)
GP_u = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
GP_u.fit(pComp.T, uComp.T)
joblib.dump(GP_u, "../results/Offline/GP_u.pkl")
GP_g = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
GP_g.fit(pComp.T, gComp.T)
joblib.dump(GP_g, "../results/Offline/GP_g.pkl")
    
toc = timeit.default_timer()
print("OFFLINE COMPUTATION TIME: "+str((toc-tic)/60.)+" min")

OFFLINE COMPUTATION TIME: 0.029813521183192884 min


In [58]:
h_skins = ipywidgets.FloatSlider(value=20e-3,min=1.45e-3,max=30e-3,step=0.001,description='h_skins:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_ribs = ipywidgets.FloatSlider(value=10e-3,min=1.45e-3,max=15e-3,step=0.001,description='h_ribs:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_le = ipywidgets.FloatSlider(value=50e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_le:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_te = ipywidgets.FloatSlider(value=40e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_te:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
b = ipywidgets.FloatSlider(value=55,min=34,max=80,step=0.1,description='b:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.1f',)
S = ipywidgets.FloatSlider(value=480,min=122,max=845,step=1,description='S:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,)
ui = ipywidgets.HBox([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])

def f5(h_skins,h_ribs,h_spars_le,h_spars_te,b,S):
    tic = timeit.default_timer()
    name_u = joblib.load("../results/Offline/GP_u.pkl")
    u_mp, u_vp = name_u.predict(np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S]).reshape(1,6), eval_MSE=True)
    name_g = joblib.load("../results/Offline/GP_g.pkl")
    g_mp, g_vp = name_g.predict(np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S]).reshape(1,6), eval_MSE=True)
    param_data = np.load("../results/Offline/param_data.npy")
    # 2) Calculation of u_est and g_est
    u_mp = u_mp[0,:]
    u_est = u_mp
    u_est1 = u_mp+3.0*np.sqrt(u_vp)
    u_est2 = u_mp-3.0*np.sqrt(u_vp)
    g_mp = g_mp[0,:]
    g_est = g_mp
    g_est1 = g_mp+3.0*np.sqrt(g_vp)
    g_est2 = g_mp-3.0*np.sqrt(g_vp)
    toc = timeit.default_timer()
    print("ONLINE COMPUTATION TIME: "+str(toc-tic)+" s")
    
    
    x_conf = np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])
    pC = np.zeros((6))
    pC[0] = param_data[5] = h_skins
    pC[1] = param_data[6] = h_ribs
    pC[2] = param_data[7] = h_spars_le
    pC[3] = param_data[8] = h_spars_te
    pC[4] = b
    pC[5] = S
    # 3) Publication of the results: u_est, g_est
    a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
    a_new.new_wing_mesh(b,S)
    vlm_mesh = '../mesh/param_wing/new_VLM.msh'
    fem_mesh = '../mesh/param_wing/new_FEM.msh'
    n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
    # 3a) Computation of strains and stress
    element_property,material,element_type = foff.create_dict(param_data)
    my_fem = FEM_study(fem_mesh,element_type,element_property,material)
    my_fem.read_mesh_file()
    strain_dict, stress_dict = my_fem.get_strain_and_stress(u_est)
    # 3b) u_est
    my_fem.post_processing(u_est,"../results/Param_wing/u_est_tf1")
    my_fem.post_processing_var1(u_est1,"../results/Param_wing/u_est1_tf1")
    my_fem.post_processing_var2(u_est2,"../results/Param_wing/u_est2_tf1")
    ## 3c) Von Mises stress
    Von_Mises_Stress_r = my_fem.get_Von_Mises(stress_dict)
    my_fem.post_processing_Von_Mises(Von_Mises_Stress_r,'../results/param_wing/result_VM_iter_tf1')
    print("Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress_r))+" Pa")
    # 3d) g_est
    my_vlm = VLM_study(vlm_mesh)
    my_vlm.post_processing_gamma('Param_wing/g_est_tf1',g_est)
    my_vlm.post_processing_gamma_var1('Param_wing/g_est1_tf1',g_est1)
    my_vlm.post_processing_gamma_var2('Param_wing/g_est2_tf1',g_est2)
    
    # Reference
    u,g=aero_struct(h_skins,h_ribs,h_spars_le,h_spars_te,b,S)
#     a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
#     a_new.new_wing_mesh(b,S)
#     vlm_mesh = '../mesh/param_wing/new_VLM.msh'
#     fem_mesh = '../mesh/param_wing/new_FEM.msh'
#     n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
#     # 3a) Computation of strains and stress
#     element_property,material,element_type = foff.create_dict(param_data)
#     my_fem = FEM_study(fem_mesh,element_type,element_property,material)
#     my_fem.read_mesh_file()
    strain_dict, stress_dict = my_fem.get_strain_and_stress(u)
    ## 3c) Von Mises stress
    Von_Mises_Stress = my_fem.get_Von_Mises(stress_dict)
    #my_fem.post_processing_Von_Mises(Von_Mises_Stress,'../results/param_wing/result_VM_iter')
    print("Reference Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress))+" Pa")
    
    Target=u_est
    res=u
    z1 = np.polyfit(Target, res, 1)
    f1 = np.poly1d(z1)
    plt.scatter(Target,res,color='r',label='Data')
    plt.plot(np.unique(Target),f1(np.unique(Target)),label='F')
    plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
    plt.title('GP U')
    plt.xlabel('Target')
    plt.ylabel('Output~='+str(z1[0])+'*Target+'+str(z1[1]))
    plt.legend()
    plt.show()
    
    Target=g_est
    res=g
    z2 = np.polyfit(Target, res, 1)
    f2 = np.poly1d(z2)
    plt.scatter(Target,res,color='r',label='Data')
    plt.plot(np.unique(Target),f2(np.unique(Target)),label='F')
    plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
    plt.title('GP G')
    plt.xlabel('Target')
    plt.ylabel('Output~='+str(z2[0])+'*Target+'+str(z2[1]))
    plt.legend()
    plt.show()
    
    rmse_u=np.sqrt(((u_est - u) ** 2).mean())
    print("RMSE U: %f" %rmse_u)
    rmse_g=np.sqrt(((g_est - g) ** 2).mean())
    print("RMSE G: %f" %rmse_g)
    #rmse_s=np.sqrt(((Von_Mises_Stress_r - Von_Mises_Stress) ** 2).mean())
    #print("RMSE Von Mises Stress: %f" %rmse_s)
    #error=(np.mean(Von_Mises_Stress_r)-np.mean(Von_Mises_Stress_r))
    error= np.linalg.norm(Von_Mises_Stress_r-Von_Mises_Stress)/np.linalg.norm(Von_Mises_Stress)
    print("Von Mises Stress Error: %f" %error)

out = ipywidgets.interactive_output(f5, {'h_skins':h_skins,'h_ribs':h_ribs,'h_spars_le':h_spars_le,'h_spars_te':h_spars_te,'b':b,'S':S})

display(ui, out)

Output()

# <a id="smt"></a> 5. SMT KPLS

In [4]:
import smt
from smt.surrogate_models.kpls import KPLS

In [6]:
uComp=np.load("../results/Offline/uComp.npy")
gComp=np.load("../results/Offline/gComp.npy")
pComp=np.load("../results/Offline/pCandidate.npy")

In [11]:
mean = "constant"
covariance = "squar_exp"
theta_U = np.array([100000.0]*6)
theta_L = np.array([0.001]*6)
theta_0 = np.array([1.0]*6)
kpls_g=KPLS( n_comp=6,poly=mean,corr=covariance,theta0 = theta_0)
kpls_g.set_training_values(pComp.T,gComp.T)
kpls_g.train()
kpls_u=KPLS( n_comp=6)#,poly=mean,corr=covariance,theta0 = theta_0)
kpls_u.set_training_values(pComp.T,uComp.T)
kpls_u.train()

___________________________________________________________________________
   
                                   KPLS
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 150
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.2209411
___________________________________________________________________________
   
                                   KPLS
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 150
   
___________________________________________________________________________
   
 Training
   
   Training ...


/Users/ydli/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:50: RuntimeWarning: invalid value encountered in true_divide
  x_weights = np.dot(X.T, y_score) / np.dot(y_score.T, y_score)
/Users/ydli/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/Users/ydli/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:290: RuntimeWarning: invalid value encountered in less
  if np.all(np.dot(Yk.T, Yk) < np.finfo(np.double).eps):
/Users/ydli/anaconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:298: RuntimeWarning: invalid value encountered in less
  Yk_mask = np.all(np.abs(Yk) < 10 * Y_eps, axis=0)


LinAlgError: SVD did not converge

In [55]:
h_skins = ipywidgets.FloatSlider(value=20e-3,min=1.45e-3,max=30e-3,step=0.001,description='h_skins:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_ribs = ipywidgets.FloatSlider(value=10e-3,min=1.45e-3,max=15e-3,step=0.001,description='h_ribs:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_le = ipywidgets.FloatSlider(value=50e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_le:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
h_spars_te = ipywidgets.FloatSlider(value=40e-3,min=4.35e-3,max=90e-3,step=0.001,description='h_spars_te:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.4f',)
b = ipywidgets.FloatSlider(value=55,min=34,max=80,step=0.1,description='b:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,readout_format='.1f',)
S = ipywidgets.FloatSlider(value=480,min=122,max=845,step=1,description='S:',
                           disabled=False,continuous_update=False,orientation='horizontal',
                           readout=True,)
ui = ipywidgets.HBox([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])

def f5(h_skins,h_ribs,h_spars_le,h_spars_te,b,S):
    tic = timeit.default_timer()
    g_est=kpls_g.predict_values(np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S]).reshape(1,6))
    toc = timeit.default_timer()
    print("ONLINE COMPUTATION TIME: "+str(toc-tic)+" s")
    
    param_data = np.load("../results/Offline/param_data.npy")
    x_conf = np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])
    pC = np.zeros((6))
    pC[0] = param_data[5] = h_skins
    pC[1] = param_data[6] = h_ribs
    pC[2] = param_data[7] = h_spars_le
    pC[3] = param_data[8] = h_spars_te
    pC[4] = b
    pC[5] = S
    # 3) Publication of the results: u_est, g_est
    a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
    a_new.new_wing_mesh(b,S)
    vlm_mesh = '../mesh/param_wing/new_VLM.msh'
    fem_mesh = '../mesh/param_wing/new_FEM.msh'
    n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
    # 3a) Computation of strains and stress
    element_property,material,element_type = foff.create_dict(param_data)
    my_fem = FEM_study(fem_mesh,element_type,element_property,material)
    my_fem.read_mesh_file()
#     strain_dict, stress_dict = my_fem.get_strain_and_stress(u_est)
#     # 3b) u_est
#     my_fem.post_processing(u_est,"../results/Param_wing/u_est_tf1")
#     my_fem.post_processing_var1(u_est1,"../results/Param_wing/u_est1_tf1")
#     my_fem.post_processing_var2(u_est2,"../results/Param_wing/u_est2_tf1")
#     ## 3c) Von Mises stress
#     Von_Mises_Stress_r = my_fem.get_Von_Mises(stress_dict)
#     my_fem.post_processing_Von_Mises(Von_Mises_Stress_r,'../results/param_wing/result_VM_iter_tf1')
#     print("Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress_r))+" Pa")
#     # 3d) g_est
#     my_vlm = VLM_study(vlm_mesh)
#     my_vlm.post_processing_gamma('Param_wing/g_est_tf1',g_est)
#     my_vlm.post_processing_gamma_var1('Param_wing/g_est1_tf1',g_est1)
#     my_vlm.post_processing_gamma_var2('Param_wing/g_est2_tf1',g_est2)
    
    # Reference
    u,g=aero_struct(h_skins,h_ribs,h_spars_le,h_spars_te,b,S)
#     a_new = mesh_deformation_airbus('../mesh/param_wing/VLM_mesh.msh','../mesh/param_wing/FEM_mesh.msh') 
#     a_new.new_wing_mesh(b,S)
#     vlm_mesh = '../mesh/param_wing/new_VLM.msh'
#     fem_mesh = '../mesh/param_wing/new_FEM.msh'
#     n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = foff.calcule_nodes(pC)
#     # 3a) Computation of strains and stress
#     element_property,material,element_type = foff.create_dict(param_data)
#     my_fem = FEM_study(fem_mesh,element_type,element_property,material)
#     my_fem.read_mesh_file()
    strain_dict, stress_dict = my_fem.get_strain_and_stress(u)
    ## 3c) Von Mises stress
    Von_Mises_Stress = my_fem.get_Von_Mises(stress_dict)
    #my_fem.post_processing_Von_Mises(Von_Mises_Stress,'../results/param_wing/result_VM_iter')
#     print("Reference Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress))+" Pa")


#     Target=u_est
#     res=u
#     z1 = np.polyfit(Target, res, 1)
#     f1 = np.poly1d(z1)
#     plt.scatter(Target,res,color='r',label='Data')
#     plt.plot(np.unique(Target),f1(np.unique(Target)),label='F')
#     plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
#     plt.title('POD U')
#     plt.xlabel('Target')
#     plt.ylabel('Output~='+str(z1[0])+'*Target+'+str(z1[1]))
#     plt.legend()
#     plt.show()
    
    Target=g_est
    res=g
    z2 = np.polyfit(Target, res, 1)
    f2 = np.poly1d(z2)
    plt.scatter(Target,res,color='r',label='Data')
    plt.plot(np.unique(Target),f2(np.unique(Target)),label='F')
    plt.plot(np.unique(Target),np.unique(Target),label='Y=T')
    plt.title('POD U')
    plt.xlabel('Target')
    plt.ylabel('Output~='+str(z2[0])+'*Target+'+str(z2[1]))
    plt.legend()
    plt.show()
#     rmse_u=np.sqrt(((u_est - u) ** 2).mean())
#     print("RMSE U: %f" %rmse_u)
    rmse_g=np.sqrt(((g_est - g) ** 2).mean())
    print("RMSE G: %f" %rmse_g)
    #rmse_s=np.sqrt(((Von_Mises_Stress_r - Von_Mises_Stress) ** 2).mean())
    #print("RMSE Von Mises Stress: %f" %rmse_s)
    #error=(np.mean(Von_Mises_Stress_r)-np.mean(Von_Mises_Stress_r))
#     error= np.linalg.norm(Von_Mises_Stress_r-Von_Mises_Stress)/np.linalg.norm(Von_Mises_Stress)
#     print("Von Mises Stress Error: %f" %error)

out = ipywidgets.interactive_output(f5, {'h_skins':h_skins,'h_ribs':h_ribs,'h_spars_le':h_spars_le,'h_spars_te':h_spars_te,'b':b,'S':S})

display(ui, out)

Output()

# References
1. LU LU, XUHUI MENG, ZHIPING MAO, and GEORGE EM KARNIADAKIS. DEEPXDE: A DEEP LEARNING LIBRARY FOR SOLVING DIFFERENTIAL EQUATIONS.
2. Sunil Suram. Reduced Order Modeling using TensorFlow.
3. Sunil Suram, Kenneth M. Bryden. Integrating a reduced-order model server into the engineering designprocess.  Advances in Engineering Software 90 (2015) 169–182.
4. https://github.com/mid2SUPAERO/POD-K_v2-CHANDRE